### Data Collection

In [1]:
from functions import *

### 1. Import queries

In [7]:
import pandas as pd

# read the excel file
excel_data = pd.read_excel('PIMS Sample Prompts.xlsx')

queries = []
for index, row in excel_data.iterrows():
    # Process each row
    queries.append(row['Prompt'])
    
queries = queries[:4]
queries = pd.DataFrame(queries, columns = ['Query'])
print(queries)



                                               Query
0            Vessel caught misreporting catch amount
1              Vessel caught falsifying fishing logs
2         Vessel caught with incorrect catch reports
3  Vessel caught underreporting catch in North At...


### 2. Data frame initialization

In [8]:
numbers_and_dates = pd.DataFrame(columns = [
    'Query', 'Engine',
    'results', 'dates', 'days'
])

search_results_articles = pd.DataFrame(columns = ['Query', 'Engine','Title', 'Dates', 'Link'])
print(numbers_and_dates)


Empty DataFrame
Columns: [Query, Engine, results, dates, days]
Index: []


### 3. Google, Bing news search API

In [9]:
from serpapi import GoogleSearch
# API Scrap functions

def google_news_search(query, location = 'Canada'):
    params = {
        "api_key": "7020efc655aafd1998bfad82b8f96fee27be3cdb26c1bb67909ba7042c56ebd1",
        "q": query,
        "tbm": "nws",
        "location": location,
        "num": "30"
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    news_results = results["news_results"]
    return news_results

def bing_news_search(query, location = "CA"):
    params = {
      "api_key": "7020efc655aafd1998bfad82b8f96fee27be3cdb26c1bb67909ba7042c56ebd1",
      "engine": "bing_news",
      "q": query,
      "cc": location,
      "count": "30"
    }
    search = GoogleSearch(params)
    results = search.get_dict()

    news_results = results["organic_results"]
    return news_results

    
def duckduckgo_news_search(query, location = "ca-en"):
    params = {
        "engine": "duckduckgo",
        "q": query,
        "kl": location,
        "tbm": "nws",
        #"api_key": "7020efc655aafd1998bfad82b8f96fee27be3cdb26c1bb67909ba7042c56ebd1"
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    print(results)
#    news_results = results["news_results"]
    
    return results


#duckduckgo_news_search("vessel underreport")

   

### 4. Search and scrap the results (title, dates, link)

In [5]:
from datetime import datetime
def convert_day(text):
    if None:
        return None
    
    pattern1 = r'^\d{4}-\d{2}-\d{2}$'
    pattern2 = r'\b\d+[hdwmy]'
    pattern3 = r'^\d{2}/\d{2}/\d{4}$'
    pattern4 = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2},\s+\d{4}'
    pattern5 = r'\b\d+(?:hrs|day|mon|wks|yrs)\b'

    if re.match(pattern1, text):
        date = datetime.strptime(text, "%Y-%M-%d").date()
        today = datetime.combine(date.today(), datetime.min.time())
        days = str(today - date).days
        return int(days[0])
    pattern2match = re.findall(pattern2, text)
    pattern5match = re.findall(pattern5,text)
    if pattern2match:
        text = pattern2match[0]
        if text[len(text)-1] == 'h':
            return 1
        if text[len(text)-1] == 'd':
            return int(text[:len(text)-1])
        if text[len(text)-1] == 'w':
            return int(text[:len(text)-1])*7
        if text[len(text)-1] == 'm':
            return int(text[:len(text)-1])*30
        if text[len(text)-1] == 'y':
            return int(text[:len(text)-1])*365    
        
    pattern5match = re.findall(pattern5,text)
    if pattern5match:
        text = pattern5match[0]
        if text[len(text)-1] == 'hrs':
            return 1
        if text[len(text)-1] == 'day':
            return int(text[:len(text)-1])
        if text[len(text)-1] == 'wks':
            return int(text[:len(text)-1])*7
        if text[len(text)-1] == 'mon':
            return int(text[:len(text)-1])*30
        if text[len(text)-1] == 'yrs':
            return int(text[:len(text)-1])*365   
        
    pattern3match = re.match(pattern3, text)
    if pattern3match:
        date_format = "%m/%d/%Y"
        try:
            # Parse the input date string into a datetime object
            given_date = datetime.strptime(text, date_format).date()
            # Get today's date
            today = date.today()
            # Calculate the difference in days
            days_difference = (today - given_date).days
            return int(days_difference)
        except ValueError:
            # Handle invalid date format or other errors
            return None
        
    pattern4match = re.match(pattern4, text)
    if pattern4match:
        date_format = '%b %d, %Y'
        try:
            # Parse the input date string into a datetime object
            given_date = datetime.strptime(text, date_format).date()
            # Get today's date
            today = datetime.now().date()
            # Calculate the difference in days
            days_difference = (today - given_date).days
            return int(days_difference)

        except ValueError:
            # Handle invalid date format or other errors
            return None
    txtlist = text.split()
    if len(txtlist) <2:
#        print('txtlist is too short', len(txtlist), text)
        return None
    if txtlist[1] in ['hour', 'hours']:
        return 1
    if txtlist[1] in ['day', 'days']:
        return int(txtlist[0])
    if txtlist[1] in ['week', 'weeks']:
        return int(txtlist[0])*7
    if txtlist[1] in ['month', 'months']:
        return int(txtlist[0])*30
    if txtlist[1] in ['year', 'years']:
        return int(txtlist[0])*365
    return None

convert_day('12 months ago')

360

In [6]:
search_functions = [
    google_news_search,
    #scrape_yahoo_news
    #yahoo_news_search, 
    bing_news_search,
    #duckduckgo_news_search
]
search_engine = ['Google', 
                 #'Yahoo', 
                 'Bing',
                 #'DuckDuckGo'
                ]

#Google
#search_results = dict()
#search_results = [{'position' : 1, 'title': 'title', 'date': '20240621', 'link': 'http://google.com'}]
#queries = ['query1', 'query 2']
for indq, query in queries.iterrows():
    for i in range(len(search_functions)):
        # Initialize numbers and dates data row with query and search engine
        num_dat_row = [query['Query'], search_engine[i]]
        
        # Do the search and only get top 20 
        search_results = search_functions[i](query['Query'])
        search_results = search_results[:min(20, len(search_results))]
        
        # Append  the number of articles to the num_dat_row
        num_dat_row.append(len(search_results))
        dates = []
        days = []
    
        for article in search_results:
            # search_data is a new data for the data frame search_results_articles
            # 'Engine', 'Position', 'Title', 'Dates', 'Link', 'Content'
            search_data = [
                query['Query'],
                search_engine[i],
                article['title'],
                article['date'],
                article['link']
            ]
            search_results_articles.loc[len(search_results_articles)] = search_data
            dates.append(search_data[3].strip())
            days.append(convert_day(search_data[3].strip()))
        # Dates and days information for the num_dat_rowx
        num_dat_row.append(dates)
        num_dat_row.append(days)
        
        # Add the num_dat_row to the data frame numbers_and_dates
        numbers_and_dates.loc[len(numbers_and_dates)] = num_dat_row




### Yahoo

In [55]:
###### For yahoo..
# To count the number of articles
def count_result(search_results):
    count = 0
    for index, result in enumerate(search_results, start=1):
        count += 1
    return count


In [56]:
######### Yahoo 
for indq, query in queries.iterrows():
        # Initialize numbers and dates data row with query and search engine
        num_dat_row = [query['Query'], 'Yahoo']
        
        # Do the search and only get top 20 
        search_results = scrape_yahoo_news(query['Query'])
        
        # Append  the number of articles to the num_dat_row
        count = count_result(search_results)
        print(query['Query'],count)
        num_dat_row.append(count)
        dates = []
        days = []
    
        for index, article in enumerate(search_results, start=1):
            # search_data is a new data for the data frame search_results_articles
            # 'Engine', 'Position', 'Title', 'Dates', 'Link', 'Content'
            search_data = [
                query['Query'],
                'Yahoo',
                article['title'],
                article['date'],
                article['link']
            ]
            search_results_articles.loc[len(search_results_articles)] = search_data
            dates.append(search_data[3].strip())
            days.append(convert_day(search_data[3].strip()))
        # Dates and days information for the num_dat_row
        num_dat_row.append(dates)
        num_dat_row.append(days)
        
        # Add the num_dat_row to the data frame numbers_and_dates
        numbers_and_dates.loc[len(numbers_and_dates)] = num_dat_row


    
#print(search_results_articles)

Vessel caught misreporting catch amount 0
Vessel caught falsifying fishing logs 0
Vessel caught with incorrect catch reports 6
Vessel caught underreporting catch in North Atlantic 0
Vessel caught misreporting haddock catch 0
Vessel caught with inaccurate fish size records 0
Vessel caught underreporting catch near protected area 0
Vessel caught misreporting tuna catch quantities 0
Vessel caught with unrecorded fish species 0
Vessel caught bypassing quota system 0


In [25]:
print(numbers_and_dates.head(20))
#search_results_articles.head()

                                                Query  Engine  results  \
0             Vessel caught misreporting catch amount  Google       20   
1             Vessel caught misreporting catch amount    Bing        3   
2               Vessel caught falsifying fishing logs  Google       20   
3               Vessel caught falsifying fishing logs    Bing       14   
4          Vessel caught with incorrect catch reports  Google       20   
5          Vessel caught with incorrect catch reports    Bing       14   
6   Vessel caught underreporting catch in North At...  Google       12   
7   Vessel caught underreporting catch in North At...    Bing       14   
8            Vessel caught misreporting haddock catch  Google        2   
9            Vessel caught misreporting haddock catch    Bing       14   
10    Vessel caught with inaccurate fish size records  Google       20   
11    Vessel caught with inaccurate fish size records    Bing       14   
12  Vessel caught underreporting catch

In [10]:
print(numbers_and_dates.head(20))

Empty DataFrame
Columns: [Query, Engine, results, dates, days]
Index: []


## Export the data (excel file)

In [59]:
import pandas as pd

# Example DataFrame
####### Have to have numberes_and_dates as a pandas data frame
#df = pd.DataFrame(data)

# Save DataFrame to Excel
numbers_and_dates.to_excel('numbers_and_dates.xlsx', index=False)

print("DataFrame has been saved to output.xlsx")


DataFrame has been saved to output.xlsx


In [60]:
import pandas as pd

######## Have to have search_results_articles as a pandas data frame
#df = pd.DataFrame(data)

# Save DataFrame to Excel
search_results_articles.to_excel('search_results_articles.xlsx', index=False)

print("DataFrame has been saved to output.xlsx")


DataFrame has been saved to output.xlsx
